### Important notes :

##### If you want to convert to another format change the .tiff in remove_obsolete_extenstions_from_name(). If you want to convert to czi make sure you remove the czi extension from the cuts in the same function. The imwrite in create_images_for default_channel and create_images_for_restchannels will probably also have to change.

##### The images that will be created depend on the files included in the selected pkl directory. However, they will only be created if they exist in the ROIs (note that only default channel exists in the ROIs, hence more keys are created in this snippet for the restchannels)


In [1]:
import cv2
import pickle
import os
import sys
import re
import copy
import numpy as np
from shapely.geometry import Polygon
from itertools import cycle, islice 
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askdirectory
from tkinter import messagebox
from os import listdir
from os.path import isfile, join, exists



def select_pickle_directory ():
    root = Tk()
    picklimages_directory =filedialog.askdirectory(title='Select pkl image folder')
    root.destroy()
    return picklimages_directory

def showmessages (msg_type, title, message):
    defroot = Tk()
    defroot.geometry('10x10')
    if msg_type == 'warning':
        messagebox.showwarning(title=title, message=message)
    if msg_type == 'error':
        messagebox.showerror(title=title, message=message)
    else :
        print(f'Wrong msg_type. Must be warning or error but it was {msg_type}')
    defroot.destroy()

def select_save_directory():
    def select_specific_directory_to_save():
        global directory_to_save
        directory_to_save = filedialog.askdirectory(title='Where to save segmented ROIs')  
        sdroot.destroy()
    sdroot = Tk()
    sdroot.title ('Select Save Folder')
    sdroot.resizable(False,False)
    sdroot.geometry('320x200')
    DirButton = Button(sdroot,text='Choose folder to save your segmented areas', command=select_specific_directory_to_save).place(x=41,y=80)
    sdroot.mainloop()    

def create_subdirectories () :
    """Creates one subfolder for each subregion that is registered in bregmata_and_regions.pkl.
       If the folder exists, it passes"""
    with open('Bregmata_and_regions.pkl', 'rb') as information_regions :
        pickle_with_bregma_and_regions = pickle.load(information_regions)
    if len(pickle_with_bregma_and_regions) == 2 :
        listed_regions = pickle_with_bregma_and_regions[1]
        listed_regions= list(islice(listed_regions, 30))
        listed_regions = list(set(listed_regions))
    else :
        showmessages(msg_type='error', title='Wrong bregmata_and_regions.pkl', message='Your bregmata_and_regions.pkl file is not consisted of two elements (list with bregmata and list(cycle type) with regions). Delete the afforementioned file and define bregma points and regions again. If the problem persists try creating a new workspace with this ipynb file. This program will terminate.')
        print('error found, will exit')
        sys.exit(0)
    for area_folder in listed_regions :                                                                  # we cannot iterate from regions because its cycle()
        possible_path = f'{directory_to_save}/{area_folder}'
        if os.path.exists(possible_path) == True:
            pass
        else:
            os.mkdir(possible_path)

def add_restchannels_to_ROI_coords (all_ROIs_dic):
    """The ROI coords only have keys of this format 'dddddd_20x-dd.czi-default.pkl'
       A format of 'dddddd_20x-dd.czi-restchannels.pkl' must be added to match the files in the 
       converted images folder that the user will select in select_pickle_directory()"""
    
    restchannelsROIs = {}
    for key in all_ROIs_dic.keys():
        additional = key.replace('default','restchannels')
        restchannelsROIs[additional] =all_ROIs_dic[key]
    
    return restchannelsROIs
        
def create_images_for_default_channel():
    """The ROI file contains only keys with the name of the default channel.
    This function creates images only for the default channel, that is, baswed on the ROIs coordinates file."""
    for image_file in all_files :
        try :
            ROIs_coords[image_file]
        except KeyError :
            non_found_images.append(image_file)
        
        else :
            whole_path = f'{directory}/{image_file}'
            with open (whole_path, 'rb') as image_to_read :
                initial_image = pickle.load(image_to_read)
            print(f'loaded image {image_file} which is of type {type(initial_image)}')
            if len(initial_image.shape) != 2 :
                showmessages(msg_type='warning', title ='Unexpected Dimensions', message=f'The expected dimensions are 2 (height and weight). File {image_file} has more dimensions than expected. It will not be processed')            
                continue
            for region, coords in ROIs_coords[image_file].items():
                if region == 'Subdirectory' or region == "Bregma":
                    continue
                else :
                    channel = 'DEF'
                    image = initial_image
                    polygon = Polygon(coords)
                    mask = np.zeros(image.shape[0:2], dtype=np.uint8)
                    arraypoints = [np.array(coords, dtype=np.int32)]
                    cv2.fillPoly(mask, arraypoints, (255, 255, 255))                                     # Fill the polygon region with white color
                    cropped = cv2.bitwise_and(image, image, mask=mask)
                    print(f'cropped the {region} of file {image_file} and channel {channel}')
                    try :
                        saved_bregma = ROIs_coords[image_file]['Bregma']
                    except : 
                        saved_bregma = 'Undefined'
                    filename = f'{directory_to_save}/{region}/{image_file}-Bregma{saved_bregma}'
                    finalname= remove_obsolete_extensions_from_name(filename)
                    cv2.imwrite(finalname, cropped,[cv2.IMWRITE_TIFF_COMPRESSION, 1])
                    print('saved it')
    
def create_images_for_restchannels(restchannels_dic):
    """The same coordinates will be used to extract files from restchannels"""
    list_with_all_restchannels_images = []
    for image_file in all_files :
        try :
            restchannels_dic[image_file]
        except KeyError :
            non_found_restchannel_images.append(image_file)
        
        else :
            whole_path = f'{directory}/{image_file}'
            with open (whole_path, 'rb') as image_to_read :
                list_with_images = pickle.load(image_to_read)
            print(f'loaded image {image_file} which is of type {type(list_with_images)}')
            # must be list above
            for image_index in range(len(list_with_images)) :
                image = list_with_images[image_index]
                print(f'image type is {type(image)}')
                if len(image.shape) != 2 :
                    showmessages(msg_type='warning', title ='Unexpected Dimensions', message=f'The expected dimensions are 2 (height and weight). File {image_file} has more dimensions than expected. It will not be processed')            
                    continue
                for region, coords in restchannels_dic[image_file].items():
                    if region == 'Subdirectory' or region == 'Bregma':
                        continue
                    else :
                        # read the comments for the line below. Numbers are arbitrary
                        channel = image_index + 1                                                     # practically, image index is the channel index since every image is one channel. Mind that numbers are arbitrary. Meaning that if the default channel was the channel 1, these images in restchannels will start with 0 regardless. So the user has to make out what channel 1 is and so on
                        polygon = Polygon(coords)
                        mask = np.zeros(image.shape[0:2], dtype=np.uint8)
                        arraypoints = [np.array(coords, dtype=np.int32)]
                        cv2.fillPoly(mask, arraypoints, (255, 255, 255))                              # Fill the polygon region with white color
                        cropped = cv2.bitwise_and(image, image, mask=mask)
                        print(f'cropped the {region} of file {image_file} and channel {channel}')
                        try:
                            saved_bregma = only_restchannels_dic[image_file]['Bregma']                # Bregma will be not be extracted by the ROIs_dic but by the only_restchannels_dic to bypass errorkeys 
                        except :
                            saved_bregma = 'Undefined'
                        filename =  f'{directory_to_save}/{region}/{image_file}zchannel{channel}-Bregma{saved_bregma}'     # zchannels instead of channel helps to sort manually by name inside the folder
                        finalname= remove_obsolete_extensions_from_name(filename)
                        cv2.imwrite(finalname, cropped,[cv2.IMWRITE_TIFF_COMPRESSION, 1])
                        print('saved it')

  
def remove_obsolete_extensions_from_name (name):
    """The name will include extensions like czi, pkl and restchannels that are obsolete, hence removed here.
       Finally, a .tiff extension will be added.
       Input name : str
       Output     : str"""
    
    if '.pkl' in name :                                                                                # should be anyway in all names  
        dotpkl_index = name.find('.pkl')                                                               # we want to remove .pkl from the filename
        dotpkl_length= 4                                                                               # because its .pkl                 
        if name.find('.pkl') != -1 :                                                                   # -1 means it was not found
            name = name[:dotpkl_index] + name[dotpkl_index+dotpkl_length:]                             # so it can be further processed for more cuts
    if '.czi' in name :
        czindex = name.find('.czi')
        czindex_length = 4
        if name.find('.czi') != -1 :
            name = name[:czindex] + name[czindex+czindex_length:] 
    if 'restchannels' in name :
        restindex = name.find('restchannels')
        restlength= 12
        if name.find('restchannels') != -1 :
            name = name[:restindex] + name [restindex+restlength:]
    name = f'{name}.tiff'
    
    return name
    

with open('ROI_Coordinates_DONT_delete.pkl', 'rb') as readable:
    ROIs_coords = pickle.load(readable)

select_save_directory()
directory = select_pickle_directory()
create_subdirectories()
all_files = [file for file in listdir(directory) if isfile(join(directory,file))]
non_found_images,non_found_restchannel_images = [],[]
create_images_for_default_channel()
only_restchannels_dic = add_restchannels_to_ROI_coords(ROIs_coords)
create_images_for_restchannels (only_restchannels_dic)



loaded image 427521_20x-01.czi-default.pkl which is of type <class 'numpy.ndarray'>
cropped the BLA of file 427521_20x-01.czi-default.pkl and channel DEF
saved it
cropped the LA of file 427521_20x-01.czi-default.pkl and channel DEF
saved it
cropped the CeA of file 427521_20x-01.czi-default.pkl and channel DEF
saved it
cropped the St of file 427521_20x-01.czi-default.pkl and channel DEF
saved it
loaded image 427521_20x-02.czi-default.pkl which is of type <class 'numpy.ndarray'>
cropped the BLA of file 427521_20x-02.czi-default.pkl and channel DEF
saved it
cropped the LA of file 427521_20x-02.czi-default.pkl and channel DEF
saved it
cropped the CeA of file 427521_20x-02.czi-default.pkl and channel DEF
saved it
cropped the St of file 427521_20x-02.czi-default.pkl and channel DEF
saved it
loaded image 427521_20x-01.czi-restchannels.pkl which is of type <class 'list'>
image type is <class 'numpy.ndarray'>
cropped the BLA of file 427521_20x-01.czi-restchannels.pkl and channel 1
saved it
crop

In [ ]:
# def rename_the_non_default_images_to_include_bregma(): 
#     """ Images of the non-default channels do not include bregma name before this function is executed.
#     This function will search for a patern with :
#     an indefinite number of digits followed by '_' followed by 2 undefined digits 
#     followed by 'x' followed by '-' an indefinite number of digits (Example : 427521_20x-01)
#     If it finds a filename with this pattern that includes the word bregma (could only be a default channel image),
#     it will adopt the value of that bregma"""
#     pattern = r"\d+_\d{2}x-\d+"
#     br_pattern = r"Bregma\d+"
#     for region_directory in listdir(directory_to_save) :
#         region_directory_path = f'{directory_to_save}/region_directory' 
#         all_semented_imges    = [seg_img for seg_img in listdir(region_directory_path) if isfile(join(region_directory_path,seg_img))]
#         for one_image_name in all_semented_imges:
#             if 'Bregma' in one_image_name :
#                 continue
#             else :
#                 anID = re.search(pattern, one_image_name)
#                 if anID :
#                     indexes_span = anID.span()
                    